In [1]:
from ultralytics import YOLO

# 1. 동영상 데모
model = YOLO("runs/detect/train4/weights/best.pt")

results = model.track("fire1.mp4", show = True)
# results = model.track("fire1.mp4", show = True, stream = True)

FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/train4/weights/best.pt'

In [ ]:
import cv2
from ultralytics import YOLO

# 1. 동영상 데모
model = YOLO("best.pt")

cap = cv2.VideoCapture(0)
results = None

# Loop through the video frames
while cap.isOpened():
  # Read a frame from the video
  success, frame = cap.read()
  
  if success:
    # run model on the frame, persisting tracks between frames
    results = model.track(frame, persist = True)
    
    annotated_frame = results[0].plot()

    if len(results) == False:
      print(results[-1].names)

    # cv2.imshow("FIRE TEST", results[-1])

In [5]:
cap.release()

In [6]:
results[0]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'fire', 1: 'fire_smoke', 2: 'general_smoke', 3: 'irrelevant'}
obb: None
orig_img: array([[[115, 125, 135],
        [115, 125, 135],
        [117, 127, 137],
        ...,
        [145, 144, 149],
        [144, 143, 148],
        [147, 145, 150]],

       [[116, 126, 136],
        [115, 125, 135],
        [117, 126, 133],
        ...,
        [145, 144, 149],
        [144, 143, 148],
        [145, 144, 149]],

       [[116, 126, 136],
        [115, 125, 135],
        [117, 127, 130],
        ...,
        [142, 140, 145],
        [145, 144, 149],
        [147, 145, 150]],

       ...,

       [[ 22,  21,  24],
        [ 20,  19,  21],
        [ 21,  20,  23],
        ...,
        [ 43,  43,  43],
        [ 41,  41,  41],
        [ 40,  40,  40]],

       [[ 20,  19,  21],
        [ 21,  20,  23],
        [ 20,  19,  21],
        ...,
        [ 4

In [7]:
# 전송할 데이터 구조 생성
import time
import json

def generateData():
  data = {
    "timestamp" : time.time(),
    "detections" : results[-1].names,
  }
  return data

data = generateData()

# 파이썬 딕셔너리를 json 문자열로 <str> 변환 (직렬화)
json_string = json.dumps(data, indent=2, ensure_ascii=False)

# 파이썬 딕셔너리를 json 문자열로 <str> 변환 (직렬화)
data = json.dumps(data, ensure_ascii=False)


# str 문자열화된 json 데이터를 utf-8 바이트 데이터로 변환 (인코딩)
message_bytes = data.encode('utf-8')

In [13]:
import socket

HOST = 'localhost' # 로컬호스트
PORT = 6600 # 6600번 포트 사용

def send(data):
  # 1. 소켓 생성
  with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    try:
      # 2. 서버의 해당 IP주소와 포트 번호로 연결 시도
      s.connect((HOST, PORT)) 
      print(f"클라이언트 : 서버 연결 시도")

      # 3. sendall: 서버에 데이터 전송
      s.sendall(data)
      print("\n클라이언트 : 전송 완료")

      # 4. recv : 서버로부터 응답 수신 (단순출력)
      data = s.recv(4096)
      if data:
        print(f"클라이언트 수신한 응답 : {data.decode()}")

    except ConnectionRefusedError:
      print("서버에 연결할 수 없습니다. 서버가 실행 중인지 확인하십시오.")
    finally:
      print("클라이언트 종료")

In [14]:
# 생성한 인코딩 데이터 확인
message_bytes

b'{"timestamp": 1757386939.6866887, "detections": {"0": "fire", "1": "fire_smoke", "2": "general_smoke", "3": "irrelevant"}}'

In [18]:
send(message_bytes)

클라이언트 : 서버 연결 시도

클라이언트 : 전송 완료
클라이언트 종료


KeyboardInterrupt: 